In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math

## Load data

In [2]:
with open("train.txt", "r") as f:
     train_data = f.readlines()

In [3]:
Nodes = []
Edges = []
for i in range(len(train_data)):
    #if i%100 == 0:
        #print(i)
    nodes_list = [int(n) for n in train_data[i].split()]
    for node in nodes_list:
        Nodes.append(node)
    for node in nodes_list[1:]:
        Edges.append((nodes_list[0],node))
Nodes = set(Nodes)

In [4]:
print(len(Nodes), len(Edges))

4867136 24004361


## Generate negative samples

In [5]:
import random
# random choose 20,000 nodes(same as the len of training data)
ng_line = random.sample(Nodes, 20000)

In [6]:
sE = set(Edges)

In [7]:
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [8]:
from tqdm import tqdm_notebook as tqdm

def generate_negative_samples():
    negative_samples = {}
    for l in tqdm(range(len(ng_line))):
        source = ng_line[l]
        sls = []
        for slinks in random.sample(Nodes, 1200):
            if not (source, slinks) in sE:
                sls.append(slinks)
        negative_samples[source] = sls
    return negative_samples

In [9]:
try:
    negative_samples = load_obj("train_neg")
except:
    negative_samples = generate_negative_samples
    save_obj(negative_samples, "train_neg")

In [10]:
list(negative_samples.keys())[1]

1671169

# Prepare data

In [73]:
try:
    pos_data = load_obj("pos_data")
except:
    pos_data = [[t,1] for t in Edges]
try:
    neg_data = load_obj("neg_data")
except:
    neg_data = []
    for k in negative_samples.keys():
        for S in negative_samples[k]:
            neg_data.append([(k,S), 0])

## Build graph

In [12]:
G = nx.Graph()
G.add_nodes_from(Nodes)
G.add_edges_from(Edges)

# Pre-calculate len, log(len)

In [48]:
pre_features = {}
for node in list(Nodes):
    num_neig = len(sorted(nx.all_neighbors(G, node)))
    log_neig = (1. / math.log(num_neig+1)) if num_neig != 0 else 0
    neig = sorted(nx.all_neighbors(G, node))
    pre_features[node] = [num_neig, log_neig, neig]

In [49]:
pre_features[1]

[3, 0.7213475204444817, [1247754, 2382107, 4588320]]

In [79]:
save_obj(pre_features,"pre_features")

# feature engineering

In [87]:
#Adamic-Adar similarity
def AA(Node1, Node2):
    sim = 0.0
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    #print(len(common_neighors))
    for node in common_neighors:
        sim += pre_features[node][1]
    return sim

#Jaccard
def Jaccard(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(len(n1[2])+len(n2[2])-lm)

#Cosine
def Cosine(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(len(n1[2])*len(n2[2]))

In [89]:
import copy
#Adding feature to data
def add_feature(d, feature):
    data = copy.deepcopy(d)
    for i in tqdm(range(len(data))):
        source, slink = data[i][0]
        for ff in feature:
            data[i].append(ff(source, slink))
    return data

In [92]:
# data_v1: 
# added feature: AA
l = 50000
pos_data_v1 = add_feature(pos_data[:l], [AA, Jaccard, Cosine])
neg_data_v1 = add_feature(neg_data[:l], [AA, Jaccard, Cosine])

In [223]:
neg_data_v1 = add_feature(neg_data[:l], [AA, Jaccard, Cosine])

In [224]:
len(neg_data_v1)

50000

In [225]:
neg_data_v1[1]

[(1605632, 619109), 0, 0.0, 0, 0]

# Make prediction

In [28]:
import math
def sigmoid(x):
    return (1 / (1 + math.exp(-x)))
def sigmoid_n(x):
    return ((1 / (1 + math.exp(-x))-0.5)*2)

In [29]:
with open("test-public.txt", "r") as f:
    test_data = f.readlines()
    test_data = [i.split() for i in test_data[1:]]

In [97]:
def predict(method):
    result = []
    for l in tqdm(range(len(test_data))):
        line = test_data[l]
        result.append((line[0], method(int(line[1]), int(line[2]))))
    return result

In [103]:
test_data[1000]

['1001', '1094048', '594102']

In [99]:
prediction = predict(AA)

In [176]:
# Combination of 3 method
mixture_result = []
for line in test_data:
    source = int(line[1])
    slink = int(line[2])
    try:
        aa = AA(source, slink)
        ja = Jaccard(source, slink)
        co = Cosine(source, slink)
    except:
        aa = 0
        ja = 0
        co = 0
    mixture_result.append([aa, ja, co])

In [185]:
mixture_result[:10]

[[0.0, 0, 0],
 [0.4076919571196112, 0.006259780907668232, 3.909839110120619e-05],
 [0.0, 0, 0],
 [1.2374608214883451, 0.0625, 0.0024665257223396757],
 [0.8026556776069618, 0.012072434607645875, 9.646922631680494e-05],
 [2.3557421029156194, 0.013474494706448507, 0.0008437801350048216],
 [0.09064221933626666, 0.002012072434607646, 0.00012966804979253112],
 [0.15741513279199884, 0.0078125, 0.0003734129947722181],
 [0.15832403284061786, 0.003976143141153081, 7.182360123536594e-05],
 [0.0, 0, 0]]

In [196]:
from scipy.stats import rankdata
aa = np.array(rankdata([i[0] for i in mixture_result]))
ja = np.array(rankdata([i[1] for i in mixture_result]))
co = np.array(rankdata([i[2] for i in mixture_result]))

In [209]:
# Score of combination model
com_prediction = []
order = 1
for score in list((aa + ja + co)/np.max((aa + ja + co))):
    com_prediction.append((order, score))
    order += 1

In [210]:
com_prediction[1]

(2, 0.6073129251700681)

## Save to file

In [211]:
import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Prediction']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)

In [212]:
save_prediction_to_csv(com_prediction, "Xudong")

# Neural Network

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [293]:
import keras
from scipy import spatial
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.layers import Dropout
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Transfer data structure

In [231]:
train_data = pos_data_v1[:49000]+neg_data_v1[:49000]
test_data = pos_data_v1[49000:]+neg_data_v1[49000:]

In [243]:
train_features = np.array([i[2:] for i in train_data])
train_labels = np.array([i[1] for i in train_data])

test_features = np.array([i[2:] for i in test_data])
test_labels = np.array([i[1] for i in test_data])

In [247]:
class DatasetIterator:
    """
    An iterator that returns randomized batches from a data set (with features and labels)
    """
    def __init__(self, features, labels, batch_size):
        assert(features.shape[0]==labels.shape[0])
        assert(batch_size > 0 and batch_size <= features.shape[0])
        self.features = features
        self.labels = labels
        self.num_instances = features.shape[0]
        self.batch_size = batch_size
        self.num_batches = self.num_instances//self.batch_size
        if (self.num_instances%self.batch_size!=0):
            self.num_batches += 1
        self._i = 0
        self._rand_ids = None

    def __iter__(self):
        self._i = 0
        self._rand_ids = np.random.permutation(self.num_instances)
        return self
        
    def __next__(self):
        if self.num_instances - self._i >= self.batch_size:
            this_rand_ids = self._rand_ids[self._i:self._i + self.batch_size]
            self._i += self.batch_size
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        elif self.num_instances - self._i > 0:
            this_rand_ids = self._rand_ids[self._i::]
            self._i = self.num_instances
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        else:
            raise StopIteration()

In [313]:
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(train_features, train_labels,
          epochs=50,
          batch_size=128)
score = model.evaluate(test_features, test_labels, batch_size=128)

Epoch 1/50
98000/98000 [==============================] - 3s 28us/step - loss: 0.6246 - acc: 0.5901
Epoch 2/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6053 - acc: 0.6021
Epoch 3/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6023 - acc: 0.6033
Epoch 4/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6013 - acc: 0.6036
Epoch 5/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6011 - acc: 0.6037
Epoch 6/50
98000/98000 [==============================] - 2s 25us/step - loss: 0.6009 - acc: 0.6038
Epoch 7/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6008 - acc: 0.6039
Epoch 8/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6008 - acc: 0.6038
Epoch 9/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6007 - acc: 0.6038
Epoch 10/50
98000/98000 [==============================] - 3s 26us/step - loss: 0.6006 - acc: 0.6039

In [314]:
NN_prediction = list(model.predict(np.array(mixture_result)))

In [315]:
order = 1
CN_prediction = []
for i in range(2000):
    CN_prediction.append((order, int(NN_prediction[i])))
    order += 1

In [316]:
CN_prediction[1]

(2, 1)

In [317]:
save_prediction_to_csv(CN_prediction, "Xudong")